In [2]:
from numpy import *
import re
import feedparser

In [3]:
# 词表到向量的转换函数
def loadDataSet():    # 创建一些实验样本
    posttingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], 
                    ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                    ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'], 
                    ['stop', 'posting', 'stupid', 'worthless', 'garbage'], 
                    ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'], 
                    ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]    # 1代表侮辱性文字， 0代表正常言论
    return posttingList, classVec

def createVocabList(dataSet):    # 创建一个包含在所有文档中出现的不重复词的列表，为此使用Python的set数据类型
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)    # 去重
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):    # 该函数的输入参数为词汇表及某个文档，输出的是文档向量
    returnVec = [0] * len(vocabList)    # 创建一个其中所含元素都为0的向量
    for word in inputSet:     # 遍历每个词条
        if word in vocabList:   # 如果词条存在于词汇表中，则置1
            returnVec[vocabList.index(word)] = 1
        else:
            print('the word: ' + word + 'is not in my Vocabulary!')
    return returnVec

In [4]:
listOPosts, listClasses = loadDataSet()
myVovatbList = createVocabList(listOPosts)
myVovatbList

['is',
 'love',
 'not',
 'mr',
 'quit',
 'stop',
 'take',
 'him',
 'to',
 'flea',
 'problems',
 'maybe',
 'so',
 'steak',
 'garbage',
 'worthless',
 'licks',
 'how',
 'stupid',
 'food',
 'my',
 'park',
 'has',
 'cute',
 'please',
 'posting',
 'dog',
 'buying',
 'help',
 'ate',
 'I',
 'dalmation']

In [5]:
setOfWords2Vec(myVovatbList, listOPosts[3])

[0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

In [14]:
# 朴素贝叶斯分类器训练函数
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)    # 计算训练的文档数目
    numWords = len(trainMatrix[0])     # 计算每篇文档的词条数
    pAbusive = sum(trainCategory) / float(numTrainDocs)    # 文档属于侮辱类的概率
    p0Num = ones(numWords)
    p1Num = ones(numWords)    # 创建NumPy.ones数组，词条出现数初始化为1，拉普拉斯平滑
    p0Denom = 2.0
    p1Denom = 2.0    # 分母初始化为2，拉普拉斯平滑
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:    # 统计属于侮辱类的条件概率所需的数据
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:    # 统计属于非侮辱类的条件概率所需的数据
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = log(p1Num / p1Denom)    # 取对数，防止下溢出
    p0Vect = log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive  # 返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率

In [15]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)
trainMat = []
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
p0V, p1V, pAb = trainNB0(trainMat, listClasses)
p0V

array([-2.56494936, -2.56494936, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -3.25809654, -2.15948425, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -3.25809654,
       -3.25809654, -2.56494936, -2.56494936, -3.25809654, -3.25809654,
       -1.87180218, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -2.56494936, -2.56494936])

In [18]:
# 朴素贝叶斯分类函数
def classifyNb(vec2Classify, p0Vec, p1Vec, pClass1):   # 对词进行分类
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

def testingNB():   # 测试分类效果
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)   # 创造一个合并的去重集合
    trainMat = []
    for postinDoc in listOPosts:   # 6个向量
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))    # 将一个一个样本，利用词汇表转换为词向量
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))  # 训练-计算条件概率  0-非侮辱类条件概率集  1-侮辱类条件概率集
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))    # 测试样本向量化
    print(str(testEntry) + ' classified as: ' + str(classifyNb(thisDoc, p0V, p1V, pAb)))
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(str(testEntry) + ' classified as: ' + str(classifyNb(thisDoc, p0V, p1V, pAb)))

In [19]:
testingNB()

['love', 'my', 'dalmation'] classified as: 0
['stupid', 'garbage'] classified as: 1


In [20]:
# 朴素贝叶斯词袋模型
def bagOfWord2VecMN(vocabList, inputSet):  
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

In [21]:
mySent = 'this book is the best book on Python or M.L. I have ever laid eyes upon.'
# mySent.split(' ')

In [22]:
regEx = re.compile('\\W*')
listOfTokens = regEx.split(mySent)
listOfTokens
[tok.lower() for tok in listOfTokens if len(tok) > 0]

/Users/luoyaohuan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [23]:
emailText = open('email/ham/'+ str(6) +'.txt', 'rb').read()
listOfTokens = regEx.split(str(emailText))

/Users/luoyaohuan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


带侮辱性词语垃圾邮件分类示例

In [24]:
# 文件解析及完整的垃圾邮件测试函数
def textParse(bigString):    # 接收一个大写字符串并将其解析为字符串列表
    regEx = re.compile('\\W*')
    listOfTokens = regEx.split(str(bigString))
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

def spamTest():    # 对贝叶斯垃圾邮件分类器进行自动化处理
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):   # 访问25个txt文件
        wordList = textParse(open('email/spam/'+ str(i) +'.txt', 'rb').read())    # 读取每个垃圾邮件，并将字符串转换成字符串列表
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)    # 标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('email/ham/'+ str(i) +'.txt', 'rb').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)    # 标记非垃圾邮件，0表示非垃圾邮件
    vocabList = createVocabList(docList)    # 创建词汇表，并去重
    trainingSet = list(range(50))
    testSet = []    # 创建存车训练集的索引值的列表和测试集的索引值的列表
    for i in range(10):   # 从50个邮件中，随机抽选40封电子邮件作为训练集，10封作为测试集
        randIndex = int(random.uniform(0, len(trainingSet)))    # 随机选取索引值
        testSet.append(trainingSet[randIndex])    # 添加测试集的索引值
        del(trainingSet[randIndex])    # 在训练集列表中删除添加到测试集的索引值
    trainMat = []
    trainClasses = []    # 创建训练集矩阵和训练集类别标签系向量
    for docIndex in trainingSet:    # 遍历训练集
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))    # 将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])    # 将类别添加到训练集类别标签系向量中
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))    # 训练朴素贝叶斯模型
    errorCpunt = 0    # 错误分类计数
    for docIndex in testSet:    # 遍历测试集
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])    # 测试集的词集模型
        if classifyNb(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:    # 如果分类错误的情况发生
            errorCpunt += 1    # 错误计数加1
    print('the error rate is: ' + str(float(errorCpunt)/len(testSet)))

In [38]:
spamTest()

the error rate is: 0.1


/Users/luoyaohuan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: split() requires a non-empty pattern match.
  after removing the cwd from sys.path.


In [84]:
ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')    # 需要VPN